In [2]:
# Application load
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications.vgg19 import VGG19
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

from tensorflow.python.keras.applications.vgg19 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

num_classes = 2

# path
vgg19_weights_path = '../pretrained/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_model=Sequential()

In [3]:
img_height=400
img_width=518
batch_size=32

In [4]:
# model 
# base_model=VGG19(weights='imagenet',include_top=False)

base_model=VGG19(include_top=False, weights='imagenet', input_shape=(img_height,img_width,1))

x=base_model.output
x=GlobalAveragePooling2D()(x)


my_model.add(VGG19(include_top=False, pooling='max', weights=vgg19_weights_path))
my_model.add(Dense(num_classes, activation='softmax'))
my_model.layers[0].trainable=False

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# application load


img_height=400
img_width=518
batch_size=32

data_generator_with_aug=ImageDataGenerator(preprocessing_function=preprocess_input,
                                           horizontal_flip=True,
                                           vertical_flip=True,
                                           rotation_raneg=0.1)
data_generator_no_aug=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator_with_aug.flow_from_directory('../data_split/train/',
                                                          target_size=(img_height, img_width),
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

validation_generator = data_generator_no_aug.flow_from_directory('../data_split/validation/',
                                                               target_size=(img_height, img_width),
                                                               class_mode='categorical')

ValueError: The `weights` argument should be either `None` (random initialization), `imagenet` (pre-training on ImageNet), or the path to the weights file to be loaded.

In [20]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 400, 518, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 400, 518, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 200, 259, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 259, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 129, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 129, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 64, 128)       0

In [11]:
t0_start=process_time()
epochs=100
history=my_model.fit_generator(train_generator,
                      steps_per_epoch=3,
                      epochs=epochs,
                      validation_data=validation_generator,
                      validation_steps=1)
t0_stop=process_time()
print("Elapsed time: ", t0_stop, t0_start)
print("Time for total "+str(epochs) +" epochs : "+str((t0_stop-t0_start)/600) +" min.")

Found 280 files belonging to 2 classes.
Using 224 files for training.


In [ ]:
import matplotlib .pyplot as plt
print(history.history.keys())

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper right')
plt.show()

In [ ]:
import pandas as pd
# df = pd.DataFrame(history)
# df.to_csv(index=False)
df = pd.DataFrame(history.history.values())

In [54]:
df.to_csv('../vgg19_results01.csv')

In [58]:
# img = keras.preprocessing.image.load_img(
# #       '../test_images/par1_early_008.jpg',
#            '../test_images/par1_late_061.jpg',
# #     '../test_images/wt_early_067.jpg', 
# #     '../test_images/wt_late_048.jpg', 
#     color_mode='grayscale', target_size=(img_height, img_width)
# )
# img_array = keras.preprocessing.image.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Create a batch

# predictions = model.predict(img_array)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(class_names[np.argmax(score)], 100 * np.max(score))
# )

This image most likely belongs to wt with a 53.59 percent confidence.
